# 物体検出 + 姿勢推定
## 前提
* Kernel
    * Python 3 (MXNet 1.8 Python 3.7 CPU Optimized)
* instance
    * t3.medium もしくは m5.large

##  ライブラリのバージョンアップ

In [ ]:
!pip install --upgrade pip
!pip install --upgrade mxnet gluoncv

## ライブラリ読み込み

In [ ]:
from matplotlib import pyplot as plt
from gluoncv import model_zoo, data, utils
from gluoncv.data.transforms.pose import detector_to_simple_pose, heatmap_to_coord
import cv2
import os
from glob import glob

## モデル読み込み

In [ ]:
# 深層学習のモデルを読み込み
detector = model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
pose_net = model_zoo.get_model('simple_pose_resnet18_v1b', pretrained=True)
# 物体検出はひとのみにする
detector.reset_class(["person"], reuse_weights=['person'])

## 使用するファイルとディレクトリを定義

In [ ]:
MV_FILE_NAME = './20170201.mp4'
INPUT_DIR = 'input'
OUTPUT_DIR = 'output'

## 動画を1フレームずつ画像に切り出して、`INPUT_DIR`に保存する

In [ ]:
cap = cv2.VideoCapture(MV_FILE_NAME)
os.makedirs(INPUT_DIR, exist_ok=True)
n = 0
while True:
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(os.path.join(INPUT_DIR,str(n).zfill(5)+'.png'),frame)
        n += 1
    else:
        break

## 出来上がった画像のファイルリストを取得

In [ ]:
input_file_list = sorted(glob(INPUT_DIR+'/*.png'))
print(input_file_list)

## 画像を一枚ずつ物体検出と姿勢推定して、`OUTPUT_DIR`に推論結果を出力する

In [ ]:
!rm {OUTPUT_DIR}/*.png
os.makedirs(OUTPUT_DIR, exist_ok=True)
for input_file in input_file_list:
    x, img = data.transforms.presets.ssd.load_test(input_file, short=512)
    class_IDs, scores, bounding_boxs = detector(x)
    try:
        pose_input, upscale_bbox = detector_to_simple_pose(img, class_IDs, scores, bounding_boxs)

        predicted_heatmap = pose_net(pose_input)
        pred_coords, confidence = heatmap_to_coord(predicted_heatmap, upscale_bbox)

        plt.rcParams['figure.figsize'] = (15.0, 15.0)
        ax = utils.viz.plot_keypoints(img, pred_coords, confidence,
                                      class_IDs, bounding_boxs, scores,
                                      box_thresh=0.1, keypoint_thresh=0.1)
        ax.axis("off")
        plt.ioff()
        plt.savefig(input_file.replace(INPUT_DIR,OUTPUT_DIR))
        plt.close()
    except:
        plt.rcParams['figure.figsize'] = (15.0, 15.0)
        plt.imshow(img)
        plt.axis("off")
        plt.ioff()
        plt.savefig(input_file.replace(INPUT_DIR,OUTPUT_DIR))
        plt.close()

## 推論した画像を動画にまとめる

In [ ]:
# !rm video.mp4

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('./video_20170201.mp4',fourcc, 29.97, (1080,1080))
for i,img_file_path in enumerate(sorted(glob(f'{OUTPUT_DIR}/*.png'))):
    img = cv2.imread(img_file_path)
    video.write(img)
video.release()